# Homework 2

In [30]:
import pandas as pd

df = pd.read_csv("forestfires.csv")

df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


## 1. Conduct data preprocessing.

#### Process the variable area as described in the paper.

In [31]:
import numpy as np

# assuming your DataFrame is called df and the column is 'area'
df['area_log'] = np.log(df['area'] + 1)

#### Transfer the variable day into a dummy variable for weekday/weekend.

In [32]:
weekend = ['sat', 'sun']

df['is_weekend'] = df['day'].str.lower().isin(weekend).astype(int)

#### Transfer the variable month into a categorical variable

In [33]:
season_map = {
    'dec': 'Winter', 'jan': 'Winter', 'feb': 'Winter',
    'mar': 'Spring', 'apr': 'Spring', 'may': 'Spring',
    'jun': 'Summer', 'jul': 'Summer', 'aug': 'Summer',
    'sep': 'Fall',   'oct': 'Fall',   'nov': 'Fall'
}

df['season'] = df['month'].str.lower().map(season_map)

## 2. Fit linear regression models to predict area and perform model selection

In [34]:
import pandas as pd
import itertools
import statsmodels.api as sm
import statsmodels.formula.api as smf

# -------------------------------
# Helper: Fit model & return AIC
# -------------------------------
def fit_model(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model

def model_aic(model):
    return model.aic


In [35]:
# One hot encode all categorical columns
df_model = pd.get_dummies(df, drop_first=True)

# Separate target and predictors
y = df_model['area_log']
X = df_model.drop(columns=['area_log'])


In [36]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 33 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   X              517 non-null    int64  
 1   Y              517 non-null    int64  
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    float64
 10  area           517 non-null    float64
 11  area_log       517 non-null    float64
 12  is_weekend     517 non-null    int64  
 13  month_aug      517 non-null    bool   
 14  month_dec      517 non-null    bool   
 15  month_feb      517 non-null    bool   
 16  month_jan      517 non-null    bool   
 17  month_jul      517 non-null    bool   
 18  month_jun 

#### Best Subset Selection (BM1)

In [ ]:
predictors = X.columns
best_subset_model = None
best_aic = float('inf')
best_subset_predictors = None

for k in range(1, len(predictors) + 1):
    for combo in itertools.combinations(predictors, k):
        X_subset = X[list(combo)].astype(float)   # FORCE numeric
        model = fit_model(X_subset, y)
        aic = model_aic(model)

        if aic < best_aic:
            best_aic = aic
            best_subset_model = model
            best_subset_predictors = combo

BM1 = best_subset_model
print("BM1 (Best Subset Predictors):", best_subset_predictors)
print(BM1.summary())
